## GLANCE: Global Actions In A Nutshell for Counterfactual Explainability

**GLANCE** is a versatile and adaptive framework for generating *global counterfactual explanations*. <br>
These explanations are expressed as actions that offer recourse to large population subgroups.<br> The framework aims to provide explanations and insights, ensuring that the actions benefit as many individuals as possible.

## Preliminaries

### Import Dependencies 
As usual in python, the first step is to import all necessary packages.



In [1]:
from xgboost import XGBClassifier
import pandas as pd
from glance.glance.glance import GLANCE
from utils import load_models, preprocess_datasets_kfold

## Load Data and Model to be used for explanations
This will serve as the demonstrative model, which we will then treat as a black box and apply our algorithm.
Of course, any model can be used in its place.



In [3]:
dataset = "adult"
model_name = "xgb"

data, affected_list, unaffected_list, model_list, train_dataset_list, feat_to_vary, target_name, num_features, cate_features = (
    preprocess_datasets_kfold(dataset, load_models(dataset, model_name), model_name)
)

Kfold for dataset adult
Creating fold 1:
	Accuracy: 86.57%
Creating fold 2:
	Accuracy: 86.69%
Creating fold 3:
	Accuracy: 86.80%
Creating fold 4:
	Accuracy: 86.29%
Creating fold 5:
	Accuracy: 86.77%


## GLANCE 
GLANCE is a clustering-based algorithm designed to generate global counterfactual explanations. <br>It starts by forming initial clusters and gradually merges them until the number of clusters matches the user-defined final_clusters parameter.<br> From each of these final clusters, the best action is selected, and together, these actions form the global explanation.

GLANCE framework is loaded with:
 - the model to be explained
 - number of initial clusters, 
 - number of final clusters, from each of which the best action is extracted
 - number of local counterfactuals, that the Local Counterfactual Method generates for each centroid of the initial clusters

 GLANCE algorithm allows the users to specify the number of global actions generated and serves as a tool to explain and debug ML models.

In [4]:
global_method = GLANCE(
    model_list[0],
    initial_clusters=100,
    final_clusters=3,
    num_local_counterfactuals=10,
)
global_method.fit(
    data.drop(columns=["Status"]),
    data["Status"],
    train_dataset_list[0],
    feat_to_vary,
)

In [5]:
clusters, clusters_res = global_method.explain_group(affected_list[0])

100%|██████████| 300/300 [00:01<00:00, 233.22it/s]


Action 1 
Age +7.0 
Capital-Gain +30064.6 

Effectiveness: 100.00%	Cost: 3.97


Action 2 
Capital-Gain +91826.2268292683 
Hours-Per-Week +51.36829268292682 

Effectiveness: 98.86%	Cost: 14.39


Action 3 
Capital-Gain +56312.71940298507 
Capital-Loss +2410.9582089552237 

Effectiveness: 99.69%	Cost: 11.16


TOTAL EFFECTIVENESS: 99.96%
TOTAL COST: 3.98


## GLANCE Output
GLANCE generates a set of final actions, with a focus on their overall impact when applied to the entire affected population. While each action is initially associated with a specific cluster, the key metrics we prioritize are the *Total Effectiveness* and *Total Cost* across the whole population.

- *Total Effectiveness* is the percentage of individuals that achieve the favorable outcome, if each one of the final actions is applied to the whole affected population.<br>
- *Total Cost* is calculated as the mean recourse cost of the whole set of final actions over the entire population.

Additionally, for each generated action the suggested changes are also reported, as well as the *effectiveness* and *cost* they achieve on the population of the cluster they were extracted from. More specifically:

- *Effectiveness*, for each cluster-action pair ($C$, $a$), represents the percentage of individuals in $C$ who get the favorable outcome when the action $a$ is applied.
<br>
- *Cost*, for each cluster-action pair ($C$, $a$), is the mean recourse cost computed when the action $a$ is applied to the individuals of cluster $C$.
<br>

## GLANCE Modularity
Our framework is highly **modular**, allowing users to customize various aspects of it. <br>

Specifically:
- **Choice of Local Counterfactual Methods**: Users can select from a variety of local counterfactual methods to generate candidate counterfactual explanations, such as:
    - **NearestNeighbors**: When queried to provide *k* counterfactuals for an affected individual, it retrieves the k nearest neighbors from the set of unaffected instances based on their proximity to the affected individual.
    - **Random Sampling**: To find counterfactuals for an affected instance, this method iteratively modifies its features one at a time. The process begins by randomly altering one feature at a time, generating multiple new candidate instances

- **Strategy for Selecting Actions**: Additionally, users can choose different strategies for selecting the best actions from the generated counterfactuals. This enables fine-tuning of the process, allowing for the optimal balance between effectiveness and recourse cost, based on user-defined preferences.
    - **max-eff** : Selects actions based on maximizing the effectiveness.
    - **low-cost** : Selects the action with the lowest cost that flips a sufficient number of instances.
    - **mean-act** : Selects the mean action from a set of candidate actions.

In order to use them, the user should provide the **fit** method with the **cf_generator** and **cluster_action_choice_algo** variables and choose the methods of his/hers liking.

In [6]:
global_method = GLANCE(
    model_list[0], initial_clusters=100, final_clusters=3, num_local_counterfactuals=10
)
global_method.fit(
    data.drop(columns=["Status"]),
    data["Status"],
    train_dataset_list[0],
    feat_to_vary,
    cf_generator="NearestNeighbors",
)

In [7]:
clusters, clusters_res = global_method.explain_group(affected_list[0])

100%|██████████| 470/470 [00:01<00:00, 264.72it/s]

Action 1 
Age +19.112149532710283 
Workclass = Federal-gov 
Fnlwgt -35.495327102806186 
Education = Doctorate 
Marital-Status = Married-civ-spouse 
Occupation = Prof-specialty 
Relationship = Husband 
Capital-Gain -103.95327102803738 
Capital-Loss -22.8411214953271 
Hours-Per-Week +2.233644859813083 

Effectiveness: 92.43%	Cost: 7.13


Action 2 
Age +10.897058823529413 
Fnlwgt -42.897058823524276 
Education = Bachelors 
Marital-Status = Married-civ-spouse 
Occupation = Exec-managerial 
Relationship = Husband 
Capital-Gain -15.970588235294118 
Capital-Loss -81.94117647058823 
Hours-Per-Week +24.0 

Effectiveness: 81.58%	Cost: 7.15


Action 3 
Age +7.25 
Workclass = Self-emp-inc 
Fnlwgt -13390.25 
Marital-Status = Married-civ-spouse 
Occupation = Craft-repair 
Relationship = Husband 
Capital-Gain +5178.0 
Hours-Per-Week +10.5 

Effectiveness: 100.00%	Cost: 5.82


TOTAL EFFECTIVENESS: 99.94%
TOTAL COST: 5.74
